# XGBoost

## Importing the libraries

In [1]:
import numpy as np
import pandas as pd

## Importing the dataset

In [2]:
dataset = pd.read_csv("Data.csv")

In [3]:
dataset.shape

(683, 11)

In [4]:
dataset.isna().sum()

Sample code number             0
Clump Thickness                0
Uniformity of Cell Size        0
Uniformity of Cell Shape       0
Marginal Adhesion              0
Single Epithelial Cell Size    0
Bare Nuclei                    0
Bland Chromatin                0
Normal Nucleoli                0
Mitoses                        0
Class                          0
dtype: int64

In [6]:
dataset.std()

Sample code number             620644.047655
Clump Thickness                     2.820761
Uniformity of Cell Size             3.065145
Uniformity of Cell Shape            2.988581
Marginal Adhesion                   2.864562
Single Epithelial Cell Size         2.223085
Bare Nuclei                         3.643857
Bland Chromatin                     2.449697
Normal Nucleoli                     3.052666
Mitoses                             1.732674
Class                               0.954592
dtype: float64

In [8]:
# on éllimine la 1ère colonne, trop de variance
X = dataset.iloc[:, 1:-1].values
y = dataset.iloc[:, -1].values

In [9]:
X.shape

(683, 9)

In [10]:
y.shape

(683,)

## Splitting the dataset into the Training set and Test set

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Training XGBoost on the Training set

In [30]:
from lightgbm import LGBMClassifier
classifier = LGBMClassifier(n_estimators=20, learning_rate=0.5, verbosity=-1)
classifier.fit(X_train, y_train)

LGBMClassifier(learning_rate=0.5, n_estimators=20, verbosity=-1)

## Making the Confusion Matrix

In [31]:
y_pred = classifier.predict(X_test)

In [32]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[85  2]
 [ 2 48]]


0.9708029197080292

## Applying k-Fold Cross Validation

In [33]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=10)
print("Accuracy\t:\t{:.2f} %".format(accuracies.mean()*100))
print("Std deviation\t:\t{:.2f} %".format(accuracies.std()*100))

Accuracy	:	96.16 %
Std deviation	:	2.37 %


In [25]:
from sklearn.model_selection import GridSearchCV
parameters = { 'n_estimators': np.arange(start=20, stop=180, step=10), 
              'learning_rate': np.arange(start=0.1, stop=1, step=0.1)}
grid_search = GridSearchCV(estimator=classifier, param_grid=parameters, scoring='accuracy', cv=10, n_jobs=-1)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=LGBMClassifier(verbosity=-1), n_jobs=-1,
             param_grid={'learning_rate': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                         'n_estimators': array([ 20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120, 130, 140,
       150, 160, 170])},
             scoring='accuracy')

In [34]:
print("Best accuracy\t:\t{:.10f} %".format(grid_search.best_score_*100))
print("Best parameters\t:\t", grid_search.best_params_)

Best accuracy	:	97.2592592593 %
Best parameters	:	 {'learning_rate': 0.6, 'n_estimators': 20}
